In [16]:
import pandas as pd
import numpy as np
import os

DATA_PATH ="../train/"
# 1. 데이터 로드
# train_ratings는 csv, genres는 tsv 파일입니다.

train_ratings = pd.read_csv(os.path.join(DATA_PATH,'train_ratings.csv'))
df_genres = pd.read_csv(os.path.join(DATA_PATH,'genres.tsv'),sep='\t')
train_ratings.head(5)
df_genres.head(5)
# 2. 데이터 병합 (Merge)
# train_ratings를 기준으로 df_genres를 붙입니다.
# TODO: on에는 공통 컬럼명, how에는 병합 방식을 적어주세요.
df_merged = pd.merge(train_ratings, df_genres, on='item', how='left')

# 3. 결측치 처리 (Fillna)
# 장르가 없는 경우(NaN) 튕겨내지 말고 'Unknown'으로 채워줍니다.
df_merged['genre'] = df_merged['genre'].fillna('Unknown')

# 검증
print(f"Total Log Count: {len(df_merged)}")
print(df_merged.head())

Total Log Count: 14126324
   user  item        time      genre
0    11  4643  1230782529     Action
1    11  4643  1230782529  Adventure
2    11  4643  1230782529      Drama
3    11  4643  1230782529     Sci-Fi
4    11   170  1230782534     Action


In [20]:
## 실험 설계 무엇을 input label 로 할지 나누는 작업

## 검증 전략 각 유저가 마지막으로 본 아이템 딱 1개를 정답으로 숨겨두고  나머지 과거 데이터만 가지고 맞출 수 있는지

df_sorted=df_merged.sort_values(['user','time'])

df_sorted.head()
## duplicated로 각 중복 user id중 가장 마지막을 추출함
df_test = df_sorted.drop_duplicates(subset=['user'], keep='last')
# 가장 마지막 인덱스를 통해서 드롭함
df_temp = df_sorted.drop(df_test.index)

df_val = df_temp.drop_duplicates(subset="user",keep='last')


df_train = df_temp.drop(df_val.index)
# 검증
print(f"Train: {len(df_train)}, Valid: {len(df_val)}, Test: {len(df_test)}")

Train: 14063604, Valid: 31360, Test: 31360


In [21]:
from collections import Counter
import itertools as it  # (A) 라이브러리 이름
# 윈도우를 5개로 정하나요?
def train_cooc_dict(df, window_size=5):
    # counter 이름을 왜 cooc라고 지은거지
    cooc_counts = Counter()
    
    # user로 묶는데 item을 []로 감싸면 무슨효과로 item이 시간순 리스트가 되는거죠
    user_item_list = df.groupby('user')['item'].apply(list)
    
    for items in user_item_list:
    
        # 아이템 리스트가 [A,B,C,D]
        for i in range(len(items)):
            # 현재 타깃 i에 대해서 윈도우 만큼의 애들을 자름 파이썬이라 index 오류는 안남..
            context_items = items[i+i : i+1+window_size]
            
            # 윈도우에 있는 context_items에서 이웃을 짝짔는다
            for neigbor in context_items:
                # (A) -> (B) 방향
                cooc_counts[(items[i], neigbor)] += 1
                # (B) -> (A) 방향
                cooc_counts[(neigbor,items[i])]+= 1
    return cooc_counts
cooc_dict = train_cooc_dict(df_train, window_size=5)
print(f"Top 5 Pairs: {cooc_dict.most_common(5)}")

Top 5 Pairs: [((5445, 5445), 3116), ((4306, 4306), 2968), ((8961, 8961), 2736), ((1215, 1215), 2362), ((29, 29), 2296)]
